In [66]:
from flask import Flask
from flask_restful import Resource, Api
import requests
import requests_cache
import os
import configparser
from pathlib import Path
import urllib.request
from concurrent.futures import ThreadPoolExecutor, as_completed

In [67]:
requests_cache.install_cache('epa_cache', backend='sqlite', expire_after=180000)


In [68]:
start_year=2015

In [69]:
end_year=2020

In [62]:
diff = range(start_year, end_year
all_years = list(diff)

In [70]:
all_years

[2014, 2015, 2016, 2017, 2018, 2019]

In [72]:
all_date_tuples = []
start_date = "010120{YY}"
end_date="123120{YY}"
for YY in all_years:
    date_args = (start_date.format(YY=YY), end_date.format(YY=YY))
    all_date_tuples.append(date_args)

In [73]:
all_date_tuples

[('0101202014', '1231202014'),
 ('0101202015', '1231202015'),
 ('0101202016', '1231202016'),
 ('0101202017', '1231202017'),
 ('0101202018', '1231202018'),
 ('0101202019', '1231202019')]

In [79]:
def make_urls(param):
    '''
    dirty little hack for generating a list of urls for the thread
    pool executor to hit up
    :param param:
    :return: list of strings
    '''

    date_args = generate_date_args()
    base_url = "https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58"
    param_snippet = "&param={param}".format(param=param)

    annual_urls = []

    for c in COUNTIES.values():
        county_filter_snippet = "&county={c}".format(c=c)
        for a, b in date_args:
            date_snippet="&bdate={a}&edate={b}&state=06".format(a=a, b=b)
            full_url= base_url + param_snippet + date_snippet + county_filter_snippet
            annual_urls.append(full_url)

    return annual_urls




In [80]:
make_urls(42101)

['https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2019990101&edate=2019991231&state=06&county=023',
 'https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2020000101&edate=2020001231&state=06&county=023',
 'https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2020010101&edate=2020011231&state=06&county=023',
 'https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2020020101&edate=2020021231&state=06&county=023',
 'https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2020030101&edate=2020031231&state=06&county=023',
 'https://aqs.epa.gov/data/api/sampleData/byCounty?email=carrilloreb9@gmail.com&key=indigocat58&param=42101&bdate=2020040101&edate=2020041231&state=06&county=023',
 'https://aqs.ep

In [81]:

def get_CO2_data(url):

    '''
    get one year C02 annual summary for one county
    :param url:
    :return: json
    '''

    req = urllib.request.urlopen(url)
    fullpath = Path(url)
    fname = fullpath.name
    ext = fullpath.suffix

    if not ext:
        raise RuntimeError("URL does not contain an extension")

    with open(fname, "wb") as handle:
        while True:
            chunk = req.read(1024)
            if not chunk:
                break
            handle.write(chunk)

    msg = "Finished downloading {fname}".fname(fname=fname)
    return msg



In [88]:
def download_all_C02(carbon_urls=None):

    if carbon_urls is None:
        carbon_urls = make_urls("42101", 'Sample')

    futures_list = []
    results = []

    with ThreadPoolExecutor(max_workers=13) as executor:
        for url in carbon_urls:
            futures = executor.submit(get_annual_summary_CO2, url)
            futures_list.append(futures)

        for future in futures_list:
            try:
                result = future.result(timeout=60)
                results.append({"status":"200", "data":result})
            except Exception:
                print("yikes")
                results.append({"status":"500", "data" :"yikes"})

    response = {"status": "200", "data": results}

    return response

In [86]:
def do_C02_download():

    carbon_urls = make_urls("42101")
    results = download_all_C02(carbon_urls)
    for result in results:
        result.json()

In [94]:
download_all_C02(carbon_urls)

yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes
yikes


{'status': '200',
 'data': [{'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'data': 'yikes'},
  {'status': '500', 'dat

In [90]:
def make_urls(param, report_tupe=None):
    '''
    dirty little hack for generating a list of urls for the thread
    pool executor to hit up
    :param param:
    :return: list of strings
    '''
    date_args = generate_date_args()
    if not report_tupe:
        report_tupe = "sampleData"
        
    base_url = "https://aqs.epa.gov/data/api/{report_type}byCounty?email=carrilloreb9@gmail.com&key=indigocat58"
    param_snippet = "&param={param}".format(param=param)

    all_urls = []

    for c in COUNTIES.values():
        county_filter_snippet = "&county={c}".format(c=c)
        for a, b in date_args:
            date_snippet="&bdate={a}&edate={b}&state=06".format(a=a, b=b)
            full_url= base_url + param_snippet + date_snippet + county_filter_snippet
            all_urls.append(full_url)

    return all_urls


In [93]:
carbon_urls= make_urls(42101)